# GET INDONESIAN AIS DATA

## SPARK SESSION

In [1]:
#Sedona Imports
import sedona.sql
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.core.SpatialRDD import PolygonRDD, PointRDD
from sedona.core.enums import FileDataSplitter

import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql.types as pst
from pyspark import StorageLevel
from pyspark.sql import SparkSession 
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType,LongType,StringType

In [2]:
spark = SparkSession. \
    builder. \
    appName('Emissions_Indonesia'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages'). \
    config("spark.sql.parquet.enableVectorizedReader", "false").\
    getOrCreate()

SedonaRegistrator.registerAll(spark)

True

## AIS DATA IN S3

In [3]:
save_path = "s3a://ungp-ais-data-historical-backup/user_temp/"
save_path_unique = save_path + "222011485/"

In [4]:
#read saved parquet
data = spark.read.parquet(save_path_unique + "ais-data-indonesia-2022.parquet", header=True)

# IMPORT PACKAGE

In [5]:
###Packages within Kernel by default
import geopandas as gpd
import h3
import matplotlib.pyplot as plt
from shapely.ops import transform
from shapely.geometry import Polygon
from datetime import datetime
import requests

import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame

import pandas as pd
import numpy as np
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import requests

generated new fontManager


## 1. Preproses Record AIS

In [6]:
import re
import math
from collections import Counter

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def compare_vessel_name(name_1, name_2):
    vector1 = text_to_vector(name_1)
    vector2 = text_to_vector(name_2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

compare = F.udf(lambda x,y:compare_vessel_name(x,y),T.DoubleType()) 

In [7]:
ais_data = data.distinct()

specs = spark.read.load("s3a://ungp-ais-data-historical-backup/register/ShipData.CSV",format="csv",sep=",",inferSchema="true",header="true")
specs = specs.withColumnRenamed("MaritimeMobileServiceIdentityMMSINumber","mmsi_ihs")\
                .withColumnRenamed("LRIMOShipNo","imo_ihs")\
                .withColumnRenamed("Draught","SummerDraught")

imo_match = ais_data\
                    .join(specs, (ais_data.imo == specs.imo_ihs),how="inner")\
                    .withColumn("matchBy", F.lit("imo"))

ais_ihs_left = ais_data.join(specs, (ais_data.imo == specs.imo_ihs),how="left_anti")
mmsi_match = ais_ihs_left.join(specs, (ais_ihs_left.mmsi == specs.mmsi_ihs),how="inner")

vessel_name_match = mmsi_match.withColumn("similarity", compare(F.col("vessel_name"), F.col("ShipName"))).filter(F.col("similarity")>=0.50)
vessel_name_match = vessel_name_match.withColumn("imo", F.col("imo_ihs"))\
                                        .withColumn("matchBy", F.lit("mmsi"))

match_record = imo_match.union(vessel_name_match.drop(F.col("similarity")))

In [8]:
table_filter = spark.createDataFrame([
    {"Keterangan": "Record AIS Indonesia Tahun 2022", "Jumlah record": data.count()},
    {"Keterangan": "Penghapusan duplikat", "Jumlah record": ais_data.count()},
    {"Keterangan": "Pencocokan dengan database IHS", "Jumlah record": match_record.count()}
])
table_filter.show()

+-------------+--------------------+
|Jumlah record|          Keterangan|
+-------------+--------------------+
|    219038333|Record AIS Indone...|
|    218717372|Penghapusan duplikat|
|    193917476|Pencocokan dengan...|
+-------------+--------------------+



In [9]:
table_match = spark.createDataFrame([
    {"Jumlah record": imo_match.count(), "Keterangan": "Cocok dengan IMO"},
    {"Jumlah record": mmsi_match.count(), "Keterangan": "Cocok dengan MMSI"},
    {"Jumlah record": vessel_name_match.count(), "Keterangan": "Cocok dengan Nama Kapal"}
])
table_match.show()

+-------------+--------------------+
|Jumlah record|          Keterangan|
+-------------+--------------------+
|    181377200|    Cocok dengan IMO|
|     15383616|   Cocok dengan MMSI|
|     12540276|Cocok dengan Nama...|
+-------------+--------------------+



In [10]:
table_vessel = spark.createDataFrame([
    {"Jumlah record": imo_match.dropDuplicates(["imo"]).count(), "Keterangan": "Cocok dengan IMO"},
    {"Jumlah record": vessel_name_match.dropDuplicates(["imo"]).count(), "Keterangan": "Cocok dengan MMSI dan Nama Kapal"}
])
table_vessel.show()

+-------------+--------------------+
|Jumlah record|          Keterangan|
+-------------+--------------------+
|        25057|    Cocok dengan IMO|
|         4048|Cocok dengan MMSI...|
+-------------+--------------------+



# 2. Preproses Database IHS

In [8]:
## mendapatkan kapal unik
match_vessel = match_record.dropDuplicates(["imo"])
# match_vessel.count()

## 2.1 Pengecekan Nilai Null

In [12]:
columns = ['GrossTonnage', 'Deadweight', 'LengthOverallLOA',
    'DateOfBuild', 'TEU', 'Powerkwmax', 'MainEngineModel', 'Speed', 'Speedmax', 'Speedservice', 'BreadthExtreme', 'SummerDraught', 'FuelType1Capacity',
    'FuelType2Capacity', 'LightDisplacementTonnage', 'MainEngineRPM', 'MainEngineType', 'Powerkwservice', 'PropulsionType',
    'ShiptypeLevel5', 'TotalBunkerCapacity']

match_vessel.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in columns]).show(vertical=True)

-RECORD 0------------------------
 GrossTonnage             | 0    
 Deadweight               | 0    
 LengthOverallLOA         | 0    
 DateOfBuild              | 0    
 TEU                      | 0    
 Powerkwmax               | 49   
 MainEngineModel          | 532  
 Speed                    | 0    
 Speedmax                 | 0    
 Speedservice             | 0    
 BreadthExtreme           | 0    
 SummerDraught            | 0    
 FuelType1Capacity        | 404  
 FuelType2Capacity        | 0    
 LightDisplacementTonnage | 0    
 MainEngineRPM            | 1130 
 MainEngineType           | 107  
 Powerkwservice           | 0    
 PropulsionType           | 0    
 ShiptypeLevel5           | 0    
 TotalBunkerCapacity      | 9884 



In [13]:
columns = ['GrossTonnage', 'Deadweight', 'LengthOverallLOA',
    'TEU', 'Powerkwmax', 'Speed', 'Speedmax', 'Speedservice', 'BreadthExtreme', 'SummerDraught', 'FuelType1Capacity',
    'FuelType2Capacity', 'LightDisplacementTonnage', 'MainEngineRPM', 'Powerkwservice', 'TotalBunkerCapacity']

match_vessel.select([F.count(F.when(F.col(c)==0, c)).alias(c) for c in columns]).show(vertical=True)

-RECORD 0-------------------------
 GrossTonnage             | 15    
 Deadweight               | 3043  
 LengthOverallLOA         | 250   
 TEU                      | 24837 
 Powerkwmax               | 422   
 Speed                    | 3662  
 Speedmax                 | 15002 
 Speedservice             | 4860  
 BreadthExtreme           | 11691 
 SummerDraught            | 1604  
 FuelType1Capacity        | 9596  
 FuelType2Capacity        | 11492 
 LightDisplacementTonnage | 9965  
 MainEngineRPM            | 0     
 Powerkwservice           | 19880 
 TotalBunkerCapacity      | 0     



# 3. Penyesuaian Spesifikasi Kapal Sesuai IMO GHG 4

In [9]:
headers = {'User-Agent': 'XY'}

map_vessel_type = "https://raw.githubusercontent.com/nandyarz/ais/main/map_vessel_type_imo4.json"
map_vessel_type=pd.read_json(map_vessel_type)

#Types Table
vessel_type = "https://raw.githubusercontent.com/nandyarz/ais/main/type_table.json"
vessel_type=pd.read_json(vessel_type)

#####Functions--------------------
##Cleaning punctuations from new ais_types
def clean_string(text):
    text=''.join([word for word in text if word not in string.punctuation])
    text=text.lower()
    
    return text

##base shiptypelevel5 to comapre to
base_stype=map_vessel_type.ShiptypeLevel5.unique().tolist()

##Compare similiraty higher than 50% and return respective shiptype5 value
def compare_similarity(text):
    comp=[text]+base_stype
    cleaned=list(map(clean_string,comp))
    vectors=CountVectorizer().fit_transform(cleaned)
    vectors=vectors.toarray()
    csim=cosine_similarity(vectors)

    val_com=np.max(csim[0,1:])

    if val_com>0.75:
        v_type=base_stype[np.argmax(csim[0,1:])]
    else:
        v_type=None

    return v_type   

##Imo bin finder
def bin_finder(vessel_t,value,df_in):
    try:
        bin_imo=df_in[((df_in.StandardVesselType==vessel_t)&(df_in.mindiff<=value)&(df_in.maxdiff>=value))].imo4bin.iloc[0]
    except:
        bin_imo=0
    return bin_imo

def ihs_preprocessing(df):
    columns = ['Powerkwmax', 'Speed', 'Speedmax', 'SummerDraught', 'MainEngineRPM']
    
    for c in columns:
        df[c].replace(0, np.nan, inplace=True)
        df[c]=df.groupby(["StandardVesselType", "imobin"])[c].apply(lambda x:x.fillna(x.mean()))
    return df

###++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
###Unit of cargo measurement per vessel type
unit={'Bulk carrier':'Deadweight',
     'Chemical tanker':'Deadweight',
     'Container':"TEU",
     'General cargo':'Deadweight',
     'Liquified gas tanker':'GrossTonnage',
     'Oil tanker':'Deadweight',
     'Other liquids tankers':'Deadweight',
     'Ferry-pax only':'GrossTonnage',
     'Cruise':'GrossTonnage',
     'Ferry-RoPax':'GrossTonnage',
     'Refrigerated bulk':'Deadweight',
     'Ro-Ro':'Deadweight',
     'Vehicle':'GrossTonnage',
     'Yacht':'GrossTonnage',
     'Service-tug':'GrossTonnage',
     'Miscellaneous-fishing':'GrossTonnage',
     'Offshore':'GrossTonnage',
     'Service-other':'GrossTonnage',
     'Miscellaneous-other':'GrossTonnage'}

##Engine type allocation
oil_eng=['Diesel-Elec & Gas Turbine(s)','Oil Engs & Fuel Cell-Electric''Oil Eng(s), Elec-Dr, Aux Sail','Oil Engines, Geared & Elec. Dr','Oil Eng(s) & Gas Turb(s) El.Dr','Oil Eng(s) Direct Dr, Aux Sail','Oil Eng(s) Dd & Gas Turb(s) El','Oil Engines, F&S, Geared Drive','Oil Engines, Direct & Elec. Dr','Oil Engines, Elec. & Direct Dr','Oil Engine(s), Drive Unknown','Oil Engines, Elec. & Geared Dr','Oil Eng(s), Geared, Aux Sail','Oil Engs & Gas Turb(s), Geared','Oil Engine(s), Electric Drive','Oil Engine(s), Direct Drive','Oil Engine(s), Geared Drive']
sail=['Sail, Aux Petrol Eng(s) D.Dr.','Sail, Aux Oil Eng(s), Elec Dr.','Sail, Aux Oil Eng(s), Geared','Sail','Sail, Aux Oil Eng(s) Direct-Dr',]
gas_tur=['Gas Turbine(s), Electric Drive','Gas Turbine(s) Or Diesel Elec.','Gas Turbine(s) & Diesel Elec.','Gas Turbine(s), Geared Drive',]
steam=['S.Turb, Gear & Oil Eng(s), Ele','St. Turb(s) Elec Dr. Or D.E.','Steam Turbine(s), Direct Drive','Steam Recip(s) With Lp Turbine','Steam Turbine(s), Elec.Drive','Steam- & Gas-Turbines, Geared','Steam Turbine(s), Geared Drive','Steam Recip(s), Direct Drive',]


def adapted_specs_imo(df_unique_imo):
    df_unique_imo.rename(columns={"vessel_type_main":"ais_type","length":"ais_loa","width":"ais_beam"},inplace=True)

    ind=df_unique_imo.copy()

    ind=ind.assign(ShiptypeLevel5=np.where(ind.ShiptypeLevel5.isnull(),ind.ais_type,ind.ShiptypeLevel5))
    ##Remove values with Shiptypelevel5 null. Not much else to do with this records. 
    ##Remove nans before similarity check
    ind=ind[ind.ShiptypeLevel5.notnull()]
    ind=ind.assign(ShiptypeLevel5=np.where(ind.ShiptypeLevel5.isin(base_stype),ind.ShiptypeLevel5,
                                        ind.ShiptypeLevel5.apply(lambda x: compare_similarity(x))))
    ##Ensure no vessel without Standard vessel type
    ind=ind[ind.ShiptypeLevel5.notnull()]

    ##---Pending----Inputation here input from AIS(Length,Beam) and Shiptypelevel5 to have [DWT,GT]. Potential RF Regressor (missForest).

    ind=pd.merge(ind,map_vessel_type,how="left",on='ShiptypeLevel5')

    ind=ind.assign(imobin=ind.apply(lambda x: bin_finder(x.StandardVesselType,x[unit[x.StandardVesselType]],vessel_type),axis=1))

    ind=ihs_preprocessing(ind)
    
    ###Fuel allocation
    ind=ind.assign(fuel=np.where(((ind.FuelType1First=='Residual Fuel')|(ind.FuelType2Second=='Residual Fuel')),
                                np.where(((ind.PropulsionType.isin(['Steam Turbine(s), Geared Drive','S.Turb, Gear & Oil Eng(s), Ele','Steam Recip(s), Direct Drive','Steam- & Gas-Turbines, Geared','Steam Turbine(s), Elec.Drive','Steam Recip(s) With Lp Turbine','Steam Turbine(s), Direct Drive','St. Turb(s) Elec Dr. Or D.E.',]))\
                                                                &(ind.StandardVesselType=='Liquified gas tanker')),"LNG","HFO"),
                                    np.where(((ind.FuelType1First=='Distillate Fuel')&(ind.FuelType2Second=='Distillate Fuel')),"MDO",
                                    np.where(((ind.FuelType1First=='Distillate Fuel')&(ind.FuelType2Second.isin(['Yes, But Type Not Known','Not Applicable','Unknown',None]))),"MDO",
                                    np.where(((ind.FuelType1First.isin(['Yes, But Type Not Known','Not Applicable','Unknown',None]))&(ind.FuelType2Second=='Distillate Fuel')),"MDO",
                                    np.where(((ind.FuelType1First=='Coal')&(ind.FuelType2Second=='Distillate Fuel')),"MDO",
                                    np.where(((ind.FuelType1First=='Methanol')&(ind.FuelType2Second=='Distillate Fuel')),'Methanol',
                                        np.where((((ind.FuelType1First=='Residual Fuel')|(ind.FuelType2Second=='Residual Fuel'))&\
                                                ((ind.StandardVesselType=='Liquified gas tanker')&(ind.PropulsionType.isin(['Steam Turbine(s), Geared Drive','S.Turb, Gear & Oil Eng(s), Ele','Steam Recip(s), Direct Drive','Steam- & Gas-Turbines, Geared','Steam Turbine(s), Elec.Drive','Steam Recip(s) With Lp Turbine','Steam Turbine(s), Direct Drive','St. Turb(s) Elec Dr. Or D.E.',])))),'LNG',
                                        np.where(((ind.FuelType1First=='Gas Boil Off')&(ind.FuelType2Second=='Distillate Fuel')),'LNG',
                                        np.where(((ind.FuelType1First.isin(["LNG",'Lpg','Lng']))&(ind.FuelType2Second=='Distillate Fuel')),'LNG',
                                        np.where(((ind.FuelType1First.isin(["LNG",'Lpg','Lng']))&(ind.FuelType2Second.isin(['Yes, But Type Not Known','Not Applicable','Unknown',None]))),'LNG',
                                        np.where(((ind.FuelType1First.isin(['Yes, But Type Not Known','Not Applicable','Unknown',None]))&(ind.FuelType2Second.isin(["LNG",'Lpg','Lng']))),'LNG',      
                                        np.where(ind.FuelType2Second=='Gas Boil Off','LNG',
                                            np.where(((ind.FuelType1First=='Nuclear')&(ind.FuelType2Second=='Distillate Fuel')),'Nuclear',
                                            np.where(((ind.FuelType1First=='Nuclear')&(ind.FuelType2Second.isin(['Yes, But Type Not Known','Not Applicable','Unknown',None]))),'Nuclear',
                                                    np.where(((ind.FuelType1First=='Coal')&(ind.FuelType2Second.isin(['Yes, But Type Not Known','Not Applicable','Unknown',None]))),'Coal',
                                                            np.where(ind.FuelType1First=='Methanol','Methanol',                                              
                                None))))))))))))))))
                )


    ###Engine types
    ind=ind.assign(meType=np.where(ind.PropulsionType.isin(oil_eng),
                                np.where(ind.MainEngineRPM<=300,"SSD",
                                np.where(ind.MainEngineRPM.between(301,900),"MSD",
                                np.where(ind.MainEngineRPM>900,"HSD","SSD"))),
                        np.where(ind.PropulsionType.isin(['Petrol Engine(s), Direct Drive','Petrol Engine(s), Geared Drive']),"HSD",       
                        np.where(ind.PropulsionType.isin(sail),"Sail",
                        np.where(ind.PropulsionType=='Battery-Electric',"Batteries",
                        np.where(ind.PropulsionType=='Non-Propelled','Non-Propelled', 
                        "SSD"))))))

    ind=ind.assign(meType=np.where(ind.fuel=="LNG",
                                np.where(((ind.MainEngineModel.str.contains("X"))|(ind.MainEngineModel.str.contains("DF"))),"LNG-Otto-SS",
                                np.where(ind.MainEngineRPM>300,"LNG-Otto-MS",    
                                np.where(ind.MainEngineModel.str.contains("ME"),"LNG-Diesel","LNG-Otto-MS"                                       
                                ))),
                            np.where(ind.fuel=="Methanol","Methanol", 
                                ind.meType)))


    ##Gas turbines and Steam turbines conditional on former filters
    ind=ind.assign(meType=np.where(((ind.PropulsionType.isin(gas_tur))|(((ind.meType.isin(["SSD","MSD"]))&(ind.fuel=="Gas")))),"Gas Turbine",
                        np.where(ind.PropulsionType.isin(steam),"Steam Turbine",
                        ind.meType                      
                    ))
                )
    ind=ind.assign(fuel=np.where(ind.meType=="Sail","Sail",
                        np.where(ind.meType=="Non-Propelled","Non-Propelled",
                        np.where(((ind.fuel.isnull())&(ind.meType=="HSD")),"MDO",
                        np.where(((ind.fuel.isnull())&(ind.meType=="MSD")),"MDO",
                        np.where(((ind.fuel.isnull())&(ind.meType=="SSD")),"HFO",
                                ind.fuel)))))
                )

    ind=ind[['imo','mmsi', 'vessel_name', 'GrossTonnage', 'Deadweight', 'LengthOverallLOA',
    'DateOfBuild', 'TEU', 'Powerkwmax', 'MainEngineModel', 'Speed', 'Speedmax', 'Speedservice', 'BreadthExtreme', 'SummerDraught', 'FuelType1Capacity',
    'FuelType2Capacity', 'LightDisplacementTonnage', 'MainEngineRPM', 'MainEngineType', 'Powerkwservice', 'PropulsionType',
    'TotalBunkerCapacity', 'StandardVesselType', 'imobin', 'fuel', 'meType','ais_beam','ais_loa']]

    return ind

In [10]:
specs = spark.createDataFrame(adapted_specs_imo(match_vessel.toPandas()))

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/tmp/ipykernel_51/2673893709.py:51: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-

In [16]:
columns = ['Powerkwmax', 'Speed', 'Speedmax', 'SummerDraught', 'MainEngineRPM']

specs.select([F.count(F.when((F.col(c)==0)|(F.col(c).isNull()), c)).alias(c) for c in columns]).show(vertical=True)

-RECORD 0------------
 Powerkwmax    | 0   
 Speed         | 0   
 Speedmax      | 0   
 SummerDraught | 0   
 MainEngineRPM | 0   



# SAVING FILE

In [23]:
save_path = "s3a://ungp-ais-data-historical-backup/user_temp/"
save_path_unique = save_path + "222011485/"

In [24]:
#save as parquet
df.write.option("header",True).mode("overwrite").parquet(save_path_unique + "ais-ihs-indonesia-2022.parquet")

# STOP SPARK SESSION

In [25]:
spark.stop()